In [1]:
import os
import sys
import gc
import pickle
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
from typing import *
from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt

pd.set_option('max_columns', 50)
pd.set_option('max_rows', 200)
warnings.simplefilter('ignore')
sns.set()

In [2]:
from dataclasses import dataclass, field, asdict
import yaml


@dataclass
class Config:
    # General
    outdir: str = "yolov5_results/results01"
    datadir: str = "../../data/yolov5_data"
    device: str = "cuda:1"
    device_id: int = 1
    num_workers: int = 8
        
    # Training config
    epochs: int = 20
    batch_size: int = 4
    lr: float = 1e-2

    # Data config
    imgconf_file: str = '../../data/VinBigData/train.csv'
    meta_file: str = '../../data/VinBigData/train_meta.csv'
    test_meta_file: str = '../../data/VinBigData/test_meta.csv'
    imgdir_name: str = "../../data/VinBigData/png1024"
    img_size: int = 1024
    seed: int = 111
    n_splits: int = 5
       
    def update(self, param_dict: Dict) -> "Config":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self
    
    def to_yaml(self, filepath: str, width: int = 120):
        with open(filepath, 'w') as f:
            yaml.dump(asdict(self), f, width=width)

In [3]:
config = Config()
base_dir = Path().resolve()

In [4]:
def seed_everything(seed: int, device: str):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    with torch.cuda.device(device):
        torch.cuda.empty_cache()

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from pathlib import Path
import json
import re
import logging
import gc
import random
import warnings
from tqdm.notebook import tqdm
from PIL import Image
import os
from glob import glob
from joblib import Parallel, delayed
import shutil as sh
from itertools import product
from collections import OrderedDict
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from ensemble_boxes import nms, weighted_boxes_fusion
import torchvision.transforms as transforms
import albumentations as al
from albumentations import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2, ToTensor
from albumentations.core.transforms_interface import DualTransform, ImageOnlyTransform

import cv2
# import pydicom
from IPython.display import display, Image
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import torch
from torch.nn import functional as f
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader, sampler
import timm
# from efficientnet_pytorch import EfficientNet

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

logging.basicConfig(format='%(asctime)s +++ %(message)s',
                    datefmt='%d-%m-%y %H:%M:%S', level=logging.INFO)
logger = logging.getLogger()

device = torch.device(config.device if torch.cuda.is_available() else 'cpu')
logger.info(device)

28-03-21 16:52:55 +++ cuda:1


In [6]:
list_remove = [34843, 21125, 647, 18011, 2539, 22373, 12675, 7359, 20642, 5502, 19818, 5832, 28056, 28333, 20758,
               925, 43, 2199, 4610, 21306, 16677, 1768, 17232, 1378, 24949, 30203, 31410, 87, 25318, 92, 31724,
               118, 17687, 12605, 26157, 33875, 7000, 3730, 18776, 13225, 1109, 2161, 33627, 15500, 28633, 28152,
               10114, 10912, 9014,  4427, 25630, 11464, 6419, 22164, 4386, 17557, 15264, 21853, 33142, 32895, 9733,
               33010, 17493, 32128, 28802, 11658, 8841, 29557, 4802, 8591, 778, 9935, 12359, 5210, 7556, 24505, 5664,
               28670, 27820, 19359, 9817, 7800, 32934, 34098, 27931, 16074, 27308, 30645, 31029, 35697, 6199, 27065,
               1771, 14689, 31860, 1975, 29294, 2304, 34018, 23406, 26501, 26011, 2479, 32796, 25836, 3032, 31454,
               32066, 19722, 15997, 6049, 9458, 11005, 23151, 24503, 35411, 18092, 23815, 30742, 33942, 34542, 7655,
               25345, 3750, 17046, 3844, 5958, 4250, 18823, 14898, 22581, 25805, 9651, 33194, 36007, 30160, 24459,
               10838, 16544, 31252, 8053, 28487, 6208, 25244, 8470, 10089, 24813, 14769, 34305, 34047, 23366, 8049,
               13276, 22380, 32797, 32440, 11031, 18304, 33692, 21349, 26333, 34331, 9110, 21092, 34882, 35626, 10203,
               25648, 30754, 29567, 33542, 15146, 26759, 20846, 22493, 33187, 22813, 30219, 14548, 14627, 20494, 28332,
               15930, 31347, 33489, 35005, 34032, 24183, 18643, 18536, 29754, 20380, 29750, 20539, 35791, 27275, 32248]
image_remove = ['9c83d9f88170cd38f7bca54fe27dc48a', 'ac2a615b3861212f9a2ada6acd077fd9',
                'f9f7feefb4bac748ff7ad313e4a78906', 'f89143595274fa6016f6eec550442af9',
                '6c08a98e48ba72aee1b7b62e1f28e6da', 'e7a58f5647d24fc877f9cb3d051792e2',
                '8f98e3e6e86e573a6bd32403086b3707', '43d3137e74ebd344636228e786cb91b0',
                '575b98a9f9824d519937a776bd819cc4', 'ca6c1531a83f8ee89916ed934f8d4847',
                '0c6a7e3c733bd4f4d89443ca16615fc6', 'ae5cec1517ab3e82c5374e4c6219a17d',
                '064023f1ff95962a1eee46b9f05f7309', '27c831fee072b232499541b0aca58d9c',
                '0b98b21145a9425bf3eeea4b0de425e7', '7df5c81873c74ecc40610a1ad4eb2943']

In [7]:
def label_resize(org_size, img_size, *bbox):
    x0, y0, x1, y1 = bbox
    x0_new = int(np.round(x0*img_size[1]/org_size[1]))
    y0_new = int(np.round(y0*img_size[0]/org_size[0]))
    x1_new = int(np.round(x1*img_size[1]/org_size[1]))
    y1_new = int(np.round(y1*img_size[0]/org_size[0]))
    return x0_new, y0_new, x1_new, y1_new


def list_color(class_list):
    dict_color = dict()
    for classid in class_list:
        dict_color[classid] = [i/256 for i in random.sample(range(256), 3)]
    
    return dict_color


def split_df(df):
    kf = MultilabelStratifiedKFold(n_splits=GlobalConfig.fold_num,
                                   shuffle=True, random_state=GlobalConfig.seed)
    df['id'] = df.index
    annot_pivot = pd.pivot_table(df, index='image_id', columns='class_id',
                                 values='id', fill_value=0, aggfunc='count') \
    .reset_index().rename_axis(None, axis=1)
    for fold, (train_idx, val_idx) in enumerate(kf.split(annot_pivot,
                                                         annot_pivot.iloc[:, 1:train_abnormal['class_id'].nunique()])):
        annot_pivot[f'fold_{fold}'] = 0
        annot_pivot.loc[val_idx, f'fold_{fold}'] = 1
    return annot_pivot


def ensemble_multibox(boxes, scores, labels, iou_thr, sigma,
                      skip_box_thr, weights=None, method='wbf'):
    if method=='nms':
        boxes, scores, labels = nms(boxes, scores, labels,
                                    weights=weights,
                                    iou_thr=iou_thr)
    elif method=='soft_nms':
        boxes, scores, labels = soft_nms(boxes, scores, labels,
                                         weights=weights,
                                         sigma=sigma,
                                         iou_thr=iou_thr,
                                         thresh=skip_box_thr)
    elif method=='nms_weight':
        boxes, scores, labels = non_maximum_weighted(boxes, scores, labels,
                                                     weights=weights,
                                                     iou_thr=iou_thr,
                                                     skip_box_thr=skip_box_thr)
    elif method=='wbf':
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels,
                                                      weights=weights,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
    
    return boxes, scores, labels

In [8]:
train_list = glob(str(base_dir / config.imgdir_name / 'train' / '*.png'))
test_list = glob(str(base_dir / config.imgdir_name / 'test' / '*.png'))
logger.info(f'Train have {len(train_list)} file and test have {len(test_list)}')

28-03-21 16:52:55 +++ Train have 15000 file and test have 3000


In [9]:
size_df = pd.read_csv(str(base_dir / config.meta_file))
size_df.columns = ['image_id', 'h', 'w']

train_df = pd.read_csv(str(base_dir / config.imgconf_file))
train_df = train_df.merge(size_df, on='image_id', how='left')
train_df[['x_min', 'y_min']] = train_df[['x_min', 'y_min']].fillna(0)
train_df[['x_max', 'y_max']] = train_df[['x_max', 'y_max']].fillna(1)

train_df.tail()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,h,w
67909,936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,0.0,0.0,1.0,1.0,3200,2444
67910,ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,0.0,0.0,1.0,1.0,2430,1994
67911,aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,0.0,0.0,1.0,1.0,2500,2048
67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0,2040,1968
67913,5e272e3adbdaafb07a7e84a9e62b1a4c,No finding,14,R16,0.0,0.0,1.0,1.0,2500,2048


In [10]:
train_normal = train_df[train_df['class_name']=='No finding'].reset_index(drop=True)
train_normal['x_min_resize'] = 0
train_normal['y_min_resize'] = 0
train_normal['x_max_resize'] = 1
train_normal['y_max_resize'] = 1

train_abnormal = train_df[train_df['class_name']!='No finding'].reset_index(drop=True)
train_abnormal[['x_min_resize', 'y_min_resize', 'x_max_resize', 'y_max_resize']] = train_abnormal \
.apply(lambda x: label_resize(x[['h', 'w']].values, (config.img_size, config.img_size), *x[['x_min', 'y_min', 'x_max', 'y_max']].values),
       axis=1, result_type="expand")
train_abnormal['x_center'] = 0.5*(train_abnormal['x_min_resize'] + train_abnormal['x_max_resize'])
train_abnormal['y_center'] = 0.5*(train_abnormal['y_min_resize'] + train_abnormal['y_max_resize'])
train_abnormal['width'] = train_abnormal['x_max_resize'] - train_abnormal['x_min_resize']
train_abnormal['height'] = train_abnormal['y_max_resize'] - train_abnormal['y_min_resize']
train_abnormal['area'] = train_abnormal.apply(lambda x: (x['x_max_resize']-x['x_min_resize'])*(x['y_max_resize']-x['y_min_resize']), axis=1)
train_abnormal = train_abnormal[~train_abnormal.index.isin(list_remove)].reset_index(drop=True)

train_abnormal.tail()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,h,w,x_min_resize,y_min_resize,x_max_resize,y_max_resize,x_center,y_center,width,height,area
35894,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0,2880,2304,107,551,250,711,178.5,631.0,143,160,22880
35895,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0,3072,3072,388,262,446,314,417.0,288.0,58,52,3016
35896,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0,2500,2048,150,272,397,618,273.5,445.0,247,346,85462
35897,db169d0be36123bd55b866d6aa73983b,Other lesion,9,R8,6.0,670.0,272.0,1736.0,2880,2304,3,238,121,617,62.0,427.5,118,379,44722
35898,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0,2040,1968,401,491,874,658,637.5,574.5,473,167,78991


In [11]:
def Preprocess_wbf(df, size, iou_thr=0.5, skip_box_thr=0.0001):
    list_image = []
    list_boxes = []
    list_cls = []
    list_h, list_w = [], []
    new_df = pd.DataFrame()
    for image_id in tqdm(df['image_id'].unique(), leave=False):
        image_df = df[df['image_id']==image_id].reset_index(drop=True)
        h, w = image_df.loc[0, ['h', 'w']].values
        boxes = image_df[['x_min_resize', 'y_min_resize',
                          'x_max_resize', 'y_max_resize']].values.tolist()
        boxes = [[j/(size-1) for j in i] for i in boxes]
        scores = [1.0]*len(boxes)
        labels = [float(i) for i in image_df['class_id'].values]
        boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels],
                                                      weights=None,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
        list_image.extend([image_id]*len(boxes))
        list_h.extend([h]*len(boxes))
        list_w.extend([w]*len(boxes))
        list_boxes.extend(boxes)
        list_cls.extend(labels.tolist())
    list_boxes = [[int(j*(size-1)) for j in i] for i in list_boxes]
    new_df['image_id'] = list_image
    new_df['class_id'] = list_cls
    new_df['h'] = list_h
    new_df['w'] = list_w
    new_df['x_min_resize'], new_df['y_min_resize'], \
    new_df['x_max_resize'], new_df['y_max_resize'] = np.transpose(list_boxes)
    new_df['x_center'] = 0.5*(new_df['x_min_resize'] + new_df['x_max_resize'])
    new_df['y_center'] = 0.5*(new_df['y_min_resize'] + new_df['y_max_resize'])
    new_df['width'] = new_df['x_max_resize'] - new_df['x_min_resize']
    new_df['height'] = new_df['y_max_resize'] - new_df['y_min_resize']
    new_df['area'] = new_df.apply(lambda x: (x['x_max_resize']-x['x_min_resize'])\
                                  *(x['y_max_resize']-x['y_min_resize']), axis=1)
    return new_df

train_abnormal = Preprocess_wbf(train_abnormal, size=config.img_size)
train_abnormal.tail()

  0%|          | 0/4394 [00:00<?, ?it/s]

,image_id,class_id,h,w,x_min_resize,y_min_resize,x_max_resize,y_max_resize,x_center,y_center,width,height,area
23768,be53fe5a49231f1c1be020b0bdd8561f,6.0,2880,2304,197,391,297,452,247.0,421.5,100,61,6100
23769,380d07a94cc4b012812119370de47192,0.0,2368,1864,576,285,709,399,642.5,342.0,133,114,15162
23770,52951d7de2485aba8ed62629eee4d254,9.0,2880,2304,135,513,170,536,152.5,524.5,35,23,805
23771,52951d7de2485aba8ed62629eee4d254,3.0,2880,2304,321,573,717,680,519.0,626.5,396,107,42372
23772,1224f07d895107573588225f692e94f9,0.0,2264,2040,515,314,639,442,577.0,378.0,124,128,15872


In [12]:
def split_df(df):
    kf = MultilabelStratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)
    df['id'] = df.index
    annot_pivot = pd.pivot_table(df, index=['image_id'], columns=['class_id'],
                                 values='id', fill_value=0, aggfunc='count') \
    .reset_index().rename_axis(None, axis=1)
    for fold, (train_idx, val_idx) in enumerate(kf.split(annot_pivot,
                                                         annot_pivot.iloc[:, 1:(1+df['class_id'].nunique())])):
        annot_pivot[f'fold_{fold}'] = 0
        annot_pivot.loc[val_idx, f'fold_{fold}'] = 1
    return annot_pivot


fold_csv = split_df(train_abnormal)
fold_csv = fold_csv.merge(size_df, on='image_id', how='left')
fold_csv.head(10)

,image_id,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,fold_0,fold_1,fold_2,fold_3,fold_4,h,w
0,0005e8e3701dfb1dd93d53e2ff537b6e,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,3072,3072
1,0007d316f756b3fa0baea2ff514ce945,1,0,0,1,0,2,0,0,0,0,0,3,0,3,0,0,0,1,0,2880,2304
2,000d68e42b71d3eac10ccc077aba07c1,1,0,0,0,0,0,0,1,0,7,0,2,0,1,0,1,0,0,0,2880,2304
3,00150343289f317a0ad5629d5b7d9ef9,2,0,0,1,0,0,0,2,0,1,4,4,0,2,0,1,0,0,0,2508,2525
4,001d127bad87592efe45a5c7678f8b8d,0,0,1,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,0,3072,3072
5,0046f681f078851293c4e710c4466058,1,0,0,0,0,0,1,0,0,0,0,2,0,5,0,0,1,0,0,2880,2304
6,0061cf6d35e253b6e7f03940592cc35e,1,0,1,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,1,3072,2540
7,00675cd546313f912cadd4ad54415d69,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2992,2606
8,008b3176a7248a0a189b5731ac8d2e95,0,0,0,1,0,0,0,3,4,2,0,0,0,0,1,0,0,0,0,3147,2505
9,009d4c31ebf87e51c5c8c160a4bd8006,1,0,0,0,1,0,0,3,0,0,1,0,0,1,0,1,0,0,0,2890,2762


In [13]:
def create_file(df, split_df, train_file, train_folder, fold):
    
    os.makedirs(f'{train_file}/labels/train/', exist_ok=True)
    os.makedirs(f'{train_file}/images/train/', exist_ok=True)
    os.makedirs(f'{train_file}/labels/val/', exist_ok=True)
    os.makedirs(f'{train_file}/images/val/', exist_ok=True)
    
    list_image_train = split_df[split_df[f'fold_{fold}']==0]['image_id']    
    train_df = df[df['image_id'].isin(list_image_train)].reset_index(drop=True)
    val_df = df[~df['image_id'].isin(list_image_train)].reset_index(drop=True)
    
    for train_img in tqdm(train_df.image_id.unique()):
        with open(f'{train_file}/labels/train/{train_img}.txt', 'w+') as f:
            row = train_df[train_df['image_id']==train_img]\
            [['class_id', 'x_center', 'y_center', 'width', 'height']].values
            row[:, 1:] /= config.img_size
            row = row.astype('str')
            for box in range(len(row)):
                text = ' '.join(row[box])
                f.write(text)
                f.write('\n')
        sh.copy(f'{train_folder}/{train_img}.png', 
                f'{train_file}/images/train/{train_img}.png')
        
    for val_img in tqdm(val_df.image_id.unique()):
        with open(f'{train_file}/labels/val/{val_img}.txt', 'w+') as f:
            row = val_df[val_df['image_id']==val_img]\
            [['class_id', 'x_center', 'y_center', 'width', 'height']].values
            row[:, 1:] /= config.img_size
            row = row.astype('str')
            for box in range(len(row)):
                text = ' '.join(row[box])
                f.write(text)
                f.write('\n')
        sh.copy(f'{train_folder}/{val_img}.png', 
                f'{train_file}/images/val/{val_img}.png')

In [14]:
def save_chest_yaml(img_size: int, fold: int, filepath: str, width: int = 120):
    chest_dict = {
        'train': f'./chest_yolo/{img_size}_mkf_{fold + 1}/images/train/',
        'val': f'./chest_yolo/{img_size}_mkf_{fold + 1}/images/val/',

        # number of classes
        'nc': 14,

        # class names
        'names': ['Aortic enlargement', 'Atelectasis', 'Calcification',
               'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration',
               'Lung Opacity', 'Nodule/Mass', 'Other lesion',
               'Pleural effusion', 'Pleural thickening', 'Pneumothorax',
               'Pulmonary fibrosis']
    }
    with open(filepath, 'w') as f:
        yaml.dump(chest_dict, f, width=width)

In [15]:
for fold in tqdm(range(config.n_splits)):
    outdir = base_dir / config.outdir / f'fold-{fold + 1}'
    os.makedirs(str(outdir), exist_ok=True)

    create_file(train_abnormal, fold_csv, str(base_dir / 'chest_yolo' / f'{config.img_size}_mkf_{fold + 1}'), str(base_dir / config.imgdir_name / 'train'), fold)
    gc.collect()

    data_file = str(outdir / 'chest.yaml')
    save_dir = str(outdir / 'runs' / 'train')
    save_chest_yaml(img_size=config.img_size, fold=fold, filepath=data_file)


    seed_everything(seed=config.seed, device=config.device)

    !python ./yolov5/train.py --epochs $config.epochs --batch-size $config.batch_size --cfg ./yolov5/chest_yaml/yolov5x.yaml --data $data_file --weights ./yolov5/chest_yaml/yolov5x.pt --img $config.img_size --device $config.device_id --project $save_dir

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3520 [00:00<?, ?it/s]

  0%|          | 0/874 [00:00<?, ?it/s]

github: skipping check (not a git repository)
YOLOv5  torch 1.7.1+cu101 CUDA:1 (Tesla V100-PCIE-32GB, 32510.5MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='./yolov5/chest_yaml/yolov5x.yaml', data='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-1/chest.yaml', device='1', epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-1/runs/train', quad=False, rect=False, resume=False, save_dir='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-1/runs/train/exp3', single_cls=False, sync_bn=False, total_batch_size=4, weights='./

  0%|          | 0/3497 [00:00<?, ?it/s]

...

  0%|          | 0/871 [00:00<?, ?it/s]

github: skipping check (not a git repository)
YOLOv5  torch 1.7.1+cu101 CUDA:1 (Tesla V100-PCIE-32GB, 32510.5MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='./yolov5/chest_yaml/yolov5x.yaml', data='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-4/chest.yaml', device='1', epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-4/runs/train', quad=False, rect=False, resume=False, save_dir='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-4/runs/train/exp', single_cls=False, sync_bn=False, total_batch_size=4, weights='./y

  0%|          | 0/3529 [00:00<?, ?it/s]

  0%|          | 0/865 [00:00<?, ?it/s]

github: skipping check (not a git repository)
YOLOv5  torch 1.7.1+cu101 CUDA:1 (Tesla V100-PCIE-32GB, 32510.5MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='./yolov5/chest_yaml/yolov5x.yaml', data='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-5/chest.yaml', device='1', epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-5/runs/train', quad=False, rect=False, resume=False, save_dir='/home/yamaguchi-milkcocholate/VinBigData/src/VinBigData-ObjectDetection/yolov5_results/results01/fold-5/runs/train/exp', single_cls=False, sync_bn=False, total_batch_size=4, weights='./y